<a href="https://colab.research.google.com/github/sidmahurkar/BERT-Sent-Embed/blob/master/Plain_BERT_Sentence_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 778kB 3.5MB/s 
     |████████████████████████████████| 1.1MB 16.6MB/s 
     |████████████████████████████████| 890kB 23.3MB/s 
     |████████████████████████████████| 3.0MB 36.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=6dbc860c0d68b29d39b44a7f2851eb0e6f6f7388ce04f87049285e198b8a4186
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
#IMPORTS
import pandas as pd
import numpy as np
import re
import os
import random
import time

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

import transformers
from transformers import BertTokenizer, BertModel

device = ('cuda:0' if torch.cuda.is_available() else 'cpu')
print("using ", device)

using  cuda:0


In [8]:
# Load pre-trained model tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# Load pre-trained model 
model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states = True, )
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [9]:
def bert_sentence_embedding(text, model):
  
  text_with_cls = "[CLS] " + text + " [SEP]"

  # Tokenize the sentence with the BERT tokenizer
  tokenized_text = tokenizer.tokenize(text_with_cls)

  # Map the token strings to their vocabulary indeces
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

  # for tup in zip(tokenized_text, indexed_tokens):
    # print('{:<12} {:>6,}'.format(tup[0], tup[1]))

  #token_type_ids to separate one sentence from other
  segments_ids = [1] * len(tokenized_text)

  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])

  with torch.no_grad():

    outputs = model(tokens_tensor, segments_tensors)
    # contains outputs of all layers
    hidden_states = outputs[2]

    """
    Current dims:
    [# layers, # batches, # tokens, # features] 
    in our case batches = 1 as only one sentence in being passed
    """
    # use the second to last layer (better results)
    token_vecs = hidden_states[-2][0] # shape [#tokens, 768]

    # Calculate the average of all token vectors.
    sentence_embedding = torch.mean(token_vecs, dim=0)

    return sentence_embedding

In [13]:
text = "Amit Shah tested positive for COVID - 19" 
sent_embed = bert_sentence_embedding(text, model)

In [15]:
sent_embed

tensor([ 1.6303e-01, -7.5981e-01, -2.1533e-01, -3.6146e-01,  7.0163e-01,
        -6.6459e-01, -4.3734e-02, -2.1611e-01,  5.3028e-01,  4.1647e-01,
         2.8040e-01,  5.6817e-02, -2.3608e-01,  1.0412e-01,  9.3520e-02,
         4.9732e-01,  1.4725e-01, -7.7932e-02,  3.3217e-01, -2.4867e-02,
         4.4138e-02, -3.1583e-01, -1.0378e-01,  2.7765e-01,  1.1387e-01,
         2.2179e-01,  1.0922e-01,  3.6618e-02,  1.3854e-01, -3.2034e-01,
         1.7332e-01,  8.6980e-01,  4.2247e-01, -6.7053e-01, -6.7389e-02,
         4.1946e-01,  5.9469e-02,  2.5778e-01, -1.1384e-01,  5.5435e-01,
        -7.3048e-01, -7.8435e-01, -8.5927e-01, -2.8355e-01,  3.5637e-01,
         1.3662e-01,  1.4391e-03,  4.0050e-01,  5.5890e-02, -2.5840e-01,
        -3.4613e-01, -4.2568e-01, -1.7841e-01, -4.3546e-01,  5.1881e-01,
         5.0660e-01, -1.2106e-01, -5.9145e-01, -9.9089e-02,  1.9268e-01,
         3.1288e-02, -1.6663e-01, -6.3877e-02, -3.7166e-01, -7.6791e-02,
         1.5263e-01, -2.0859e-01, -1.6084e-01,  3.5

In [14]:
sent_embed.size()

torch.Size([768])